#__Stacking__
Let's understand the ensemble technique of stacking.

## Step 1: Import the Required Libraries

- Install vecstack, if it is not already installed or is giving errors while importing the library
- Import pandas, sklearn, xgboost, and vecstack libraries



In [ ]:
pip install vecstack

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from vecstack import stacking

## Step 2: Load and Preprocess the Dataset

- Load the dataset wine.data
- Add the names to the dataset as the indices
- Peek at the dataset using the head() command

In [ ]:
link = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
names = ['Class', 'Alcohol', 'Malic acid', 'Ash',
         'Alcalinity of ash' ,'Magnesium', 'Total phenols',
         'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',     'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
         'Proline']
df = pd.read_csv(link, header=None, names=names)
df.head(5)

__Observations:__
- This is the head of the dataset.
- Define X and y and split the data into train and test
- Assign the target variable class to y
- Assign all the rows and every column but the first to X
- Use the train_test_split method to split the data into an 80:20 ratio

In [ ]:
y = df[['Class']]
X = df.iloc[:,1:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

## Step 3: Define the Base Models for Stacking

- Create the models using KNeighborsClassifier, RandomForestClassifier, and XGBoostClassifier 

In [ ]:
models = [
    KNeighborsClassifier(n_neighbors=5,
                        n_jobs=-1),
        
    RandomForestClassifier(random_state=0, n_jobs=-1, 
                           n_estimators=100, max_depth=5),
        
    XGBClassifier(random_state=0, n_jobs=-1, learning_rate=0.1, 
                  n_estimators=100, max_depth=5)
]

__Observation:__
- The different classifiers are set.

## Step 4: Perform Stacking

- Let us do the S-train and S-tests, as we will be stacking the models by passing train of X and y.

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost

base_learner = [("rf", RandomForestClassifier()),
                ("log", LogisticRegression()),
                ('dtree',DecisionTreeClassifier())]
meta_learner = xgboost.XGBClassifier() 

from vecstack import StackingClassifier

stack = StackingClassifier(base_learners = base_learner,
                           meta_learner = meta_learner,
                           mode = 'oof_pred_bag',
                           n_folds=4)


In [ ]:
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
S_train, S_test = stacking(models, X_train, y_train, X_test, regression=False, mode='oof_pred_bag', 
                           needs_proba=False, save_dir=None, metric=accuracy_score, n_folds=4, 
                           stratified=True, shuffle=True, random_state=0, verbose=2)
warnings.filterwarnings("default", category=UserWarning)
warnings.filterwarnings("default", category=FutureWarning)


__Observations:__
- You can observe the accuracy score and other metrics for different models.
- For the k-nearest neighbor, the mean is 72.
- For random forest, the mean is 97.
- For the XGBoost classifier, the mean is 76. 